In [2]:
pip install --upgrade sentence-transformers pytrec_eval

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.0 MB/s eta 0

In [ ]:
import pandas as pd

df = ranks = pd.read_csv("/kaggle/input/rel-ranks/ranks.csv")
ranks.dtypes

keyword              object
average_position    float64
website              object
rel                 float64
dtype: object

In [ ]:
ranks["keyword"].isna().sum()

In [ ]:

keyword_counts = (
    ranks.groupby("keyword")["website"].nunique()
    .reset_index(name="site_count")
)


popular_keywords = keyword_counts[keyword_counts["site_count"] > 3]  # adjustable threshold
popular_keywords = popular_keywords.sort_values("site_count", ascending=False)


In [ ]:
import tqdm.auto as tqdm


selected = set()
website_groups = ranks.groupby("website")
for website, group in website_groups:
    candidates = group[group["keyword"].isin(popular_keywords["keyword"])]
    if not candidates.empty:
        # pick the keyword that appears on the most websites overall
        chosen = (
            candidates.merge(popular_keywords, on="keyword")
            .sort_values("site_count", ascending=False)
            .iloc[0]["keyword"]
        )
        selected.add(chosen)

representative_keywords = list(selected)


In [4]:
from pathlib import Path
import pickle

# Save test queries
QUERY_PATH = Path("/kaggle/input/rel-ranks/test_queries.pkl")
# QUERY_PATH.parent.mkdir(parents=True, exist_ok=True)
# with open(QUERY_PATH, "wb") as f:
#     pickle.dump(test_queries, f)


# Load test queries
with open(QUERY_PATH, "rb") as f:
    test_queries = pickle.load(f)


In [5]:
RELEVANCE_THRESHOLD = 0.1

# only keep those queries in ranks
test_df = ranks[ranks["keyword"].isin(test_queries)].copy()
test_df["is_relevant"] = test_df["rel"] >= RELEVANCE_THRESHOLD

# group by keyword and create dicts of website→rel
qrels = {
    query: group.set_index("website")["is_relevant"].to_dict()
    for query, group in test_df.groupby("keyword")
}

# groups = ranks.groupby("keyword")
# # Show the df:
# groups.size()

# qrels

In [6]:
import pytrec_eval

evaluator = pytrec_eval.RelevanceEvaluator(
    qrels, {"ndcg_cut", "map", "recall", "iprec_at_recall"}
)
# pytrec_eval.supported_measures


In [7]:
!hf auth login --token ...

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: read).
The token `kaggle eval` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `kaggle eval`


In [8]:
# MODEL = "intfloat/multilingual-e5-large"
# MODEL = "google/embeddinggemma-300m"
# MODEL = "xmanii/maux-gte-persian-v3"
# MODEL = "beademiguelperez/sentence-transformers-multilingual-e5-small"
# MODEL = "LiquidAI/LFM2-ColBERT-350M" # Failed
# MODEL = "sentence-transformers/all-MiniLM-L6-v2"
MODEL = "nvidia/llama-embed-nemotron-8b"


In [ ]:
from sentence_transformers import SentenceTransformer


model = SentenceTransformer(MODEL, trust_remote_code=True)


2025-10-31 13:01:04.453942: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761915664.642194      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761915664.695930      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/362 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/117M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
keywords = ranks["keyword"].unique()
keyword_embs = model.encode(
    keywords, show_progress_bar=True, normalize_embeddings=True, batch_size=64
)


In [ ]:
with open("/kaggle/working/keywords.txt", "w", encoding="utf-8") as f:
    f.writelines()


In [ ]:
import pandas as pd
import torch
from torch import nn
import tqdm.auto as tqdm

top_k = 32

# 4️⃣ Compute similarities and build qrels/run for PyTREC
cos = nn.CosineSimilarity(dim=1)
all_vecs = torch.tensor(keyword_embs, device="cuda")
all_keywords = pd.Series(keywords)
queries = all_keywords[all_keywords.isin(test_queries)]

run = {}  # model scores

for idx, query in tqdm.tqdm(list(queries.items())):
    run[query] = {}

    q_vec = all_vecs[idx].unsqueeze(0)
    sims = cos(q_vec, all_vecs)
    scores, indices = torch.topk(sims, k=top_k)

    scores_df = pd.DataFrame(
        {"similarity": scores.tolist()},
        index=indices.tolist(),
    )
    scores_df["keyword"] = all_keywords[scores_df.index]

    # Merge with rank info (to get websites and average positions)
    merged = scores_df.merge(
        ranks[["keyword", "website", "average_position"]], on="keyword", how="inner"
    )

    # Compute final website scores (vectorized)
    merged["score"] = merged["similarity"] / merged["average_position"]

    website_scores = (
        merged.groupby("website")["score"].mean().sort_values(ascending=False)
    )

    # Convert to PyTREC "run" format
    run[query] = website_scores.to_dict()

In [ ]:
# Evaluate
results = evaluator.evaluate(run)

In [ ]:
import json
from pathlib import Path

EVAL_DIR = Path("/kaggle/working/eval/metrics")
EVAL_DIR.mkdir(parents=True, exist_ok=True)


def save_eval_results(results_dict: dict, model_name: str):
    file = EVAL_DIR / f"{model_name.replace('/', '-')}.json"
    with open(file, "w", encoding="utf-8") as f:
        json.dump(results_dict, f, ensure_ascii=False)

    print(f"✅ Saved metrics to {file}")


save_eval_results(results, MODEL)